### Implied Volatility Model
Computed by minimizing the squarred differences between the simulated and observed prices.

In [ ]:
from scipy.optimize import minimize

# Convert the Pandas Series to a NumPy array for consistent calculations
observed_values = product_prices["value"].to_numpy()

# Function to calculate mean squared error
def mean_squared_error(simulated, observed):
    return np.mean((simulated - observed) ** 2)

# Function to simulate GBM and calculate expected value
def simulate_and_calculate_expected_value(sigma):
    GBM_expected_values = []

    for date, prices in zip(simulation_dates, backtest_windows):
        # Convert date to index (so we can lookup prices and interest rate curves)
        date_index = simulation_dates.get_loc(date)

        # Estimate r
        delta = len(product_lifetime[date:])
        tau = delta / 252
        curve_fit = curves[date_index]
        r = curve_fit(tau)/100

        # Ensure sigma is a scalar for compatibility with simulation function
        sigma_scalar = sigma.item() if isinstance(sigma, np.ndarray) else sigma

        # Simulate prices, calculate payoff and expected value for the given sigma
        simulated_prices = simulate_GBM(num_sim, prices[-1], r, sigma_scalar, delta_t, tau, Z=Z_matrix[date_index])
        payoffs = np.exp(-r*tau)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
        expected_value = np.mean(payoffs)
        GBM_expected_values.append(expected_value)

    return GBM_expected_values

# Function to minimize
def objective_function(sigma):
    simulated_values = simulate_and_calculate_expected_value(sigma)
    mse = mean_squared_error(simulated_values, observed_values)
    return mse

# Initial guess for sigma
initial_sigma = 0.2  # Adjust this based on your prior knowledge

# Run optimization to find the implied volatility
result = minimize(objective_function, initial_sigma, method='Nelder-Mead')

# The optimal sigma (implied volatility)
implied_volatility = result.x[0]

# Print or return the implied volatility
print("Implied Volatility:", implied_volatility)
print("Historical Volatility:", sigma)

Simulating using the implied volatility as computed above;

In [ ]:
# GBM without Variance Reduction and the Implied Volatility Approach
GBM_expected_values_implied_sigma = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  #delta = num_periods - date_index     # Correct this 
  delta = len(product_lifetime[date:])
  tau = delta / 252
  curve_fit = curves[date_index]
  r = curve_fit(tau)/100

  # Calulcate mu and sigma (we discard mu)
  sigma = implied_volatility

  # Simulate prices, calculate payoff and expected value
  simulated_prices = simulate_GBM(num_sim, prices[-1], r, sigma, delta_t, tau, Z=Z_matrix[date_index])
  payoffs = np.exp(-r*tau)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
  expected_value = np.mean(payoffs)
  GBM_expected_values_implied_sigma.append(expected_value)

and plotting the results:

In [ ]:
simulation_reults['Implied volatility'] = GBM_expected_values_implied_sigma

plt.figure(figsize=(14,8))
plt.title('Backtesting ',fontdict={"fontsize":18})

simulation_reults['Actual'].plot(legend=True)
simulation_reults['GBM'].plot(legend=True,style="--")
simulation_reults['Implied volatility'].plot(legend=True,style="--")

plt.xlabel('Market Days')
plt.ylabel('Derivative Price in USD')